In [3]:
import numpy as np
import pandas as pd
import glob
from skimage.io import imread
from skimage import io
from skimage.color import gray2rgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import seaborn as sns
from sklearn import datasets, preprocessing, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
from mlxtend.plotting import plot_confusion_matrix

In [4]:
# def convert_to_csv(path_to_dataset):
#     image_paths = glob.glob(path_to_dataset + '\\*\\*\\*\\*.JPEG')

#     train_imgs = []
#     test_imgs = []

#     for image_path in image_paths:
#         image = imread(image_path)
#         if image.shape[-1] != 3:
#             image = gray2rgb(image)
#         if image_path.split('\\')[-3] == 'train':
#             train_imgs.append(image.ravel().tolist() + [image_path.split('\\')[-2]])

#         else:
#             test_imgs.append(image.ravel().tolist() + [image_path.split('\\')[-2]])

#     img_0 = imread(image_paths[0])
#     column_names = [f"p_{x}" for x in range(np.prod(img_0.shape))] + ['label']

#     train_df = pd.DataFrame(train_imgs, columns=column_names)
#     test_df = pd.DataFrame(test_imgs, columns=column_names)

#     train_df.to_csv(path_to_dataset+'/train.csv', index=False)
#     test_df.to_csv(path_to_dataset+'/test.csv', index=False)

# convert_to_csv('Group_3')

In [5]:
df_train = pd.read_csv('Group_3/train.csv')

FileNotFoundError: ignored

In [ ]:
df_train.shape

In [ ]:
df_train

In [ ]:
unique_labels = df_train["label"].unique()
print(unique_labels)
print(len(unique_labels))

In [ ]:
print(df_train.label.value_counts())

In [ ]:
df_test = pd.read_csv('Group_3/test.csv')

In [ ]:
df_test.shape

In [ ]:
df_test

In [ ]:
print(df_test.label.value_counts())

In [ ]:
df_train.label.value_counts().plot(kind='bar', title='Count (label)')

In [ ]:
df_test.label.value_counts().plot(kind='bar', title='Count (label)');

In [ ]:
df_train['label']

In [ ]:
print(df_train.drop(['label'], axis=1))

In [ ]:
#converting the labels from categorical values to numerical values

lab = LabelEncoder()
lab.fit(df_train["label"])
df_train["label"] = lab.transform(df_train["label"])
df_test["label"] = lab.transform(df_test["label"])

In [ ]:
X_train = df_train.drop(['label'], axis=1)
y_train = df_train["label"]

In [ ]:
X_test= df_test.drop(['label'], axis=1)
y_test= df_test["label"]

In [ ]:
#scaling train

X_train_norm = preprocessing.minmax_scale(X_train)
df_train = pd.DataFrame(X_train_norm, columns=df_train.columns[:-1])
df_train.head()

In [ ]:
#scaling test

X_test_norm = preprocessing.minmax_scale(X_test)
df_test = pd.DataFrame(X_test_norm, columns=df_test.columns[:-1])
df_test.head()

In [ ]:
model = linear_model.LogisticRegression(solver='lbfgs', max_iter=17)
model = model.fit(X_train_norm, y_train)

y_train_predict = model.predict(X_train_norm)

y_test_predict = model.predict(X_test_norm)

acc_training = np.mean(y_train == y_train_predict) * 100
acc_testing = np.mean(y_test == y_test_predict) * 100
print("Training accuracy (%) =", format(acc_training, ".2f"))
print("Testing accuracy (%) =", format(acc_testing, ".2f"))


In [ ]:
cm_train = confusion_matrix(y_train, y_train_predict)

plt.figure(figsize=(6, 4))
sns.heatmap(cm_train, annot=True, fmt="d", cmap="Blues", cbar=True)
plt.title('Confusion Matrix - Train Set')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
cm_test = confusion_matrix(y_test, y_test_predict)

plt.figure(figsize=(6, 4))
sns.heatmap(cm_test, annot=True, fmt="d", cmap="Blues", cbar=True)
plt.title('Confusion Matrix - Test Set')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
def calc(cnf_matrix):
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    TPR = TP/(TP+FN)
    TNR = TN/(TN+FP)
    PPV = TP/(TP+FP)
    NPV = TN/(TN+FN)
    F1 = 2*((PPV)*(TPR)/(PPV + TPR))

    return TPR, TNR, PPV, NPV, F1

TPR, TNR, PPV, NPV, F1 = calc(cm_train)
print(TPR, TNR, PPV, NPV, F1)

In [ ]:
from tabulate import tabulate

print("Classification Report of Train Set:")
table_data = [
    ["Sensitivity", "Specificity", "PPV", "NPV", "F1 Score"],
]

for i in range(len(TPR)):
    table_data.append([round(TPR[i], 2), round(TNR[i], 2), round(PPV[i], 2), round(NPV[i], 2), round(F1[i], 2)])

print(tabulate(table_data, headers="firstrow", tablefmt="pretty",showindex=True))

In [ ]:
TPR, TNR, PPV, NPV, F1 = calc(cm_test)
print(TPR, TNR, PPV, NPV, F1)

In [ ]:
print("Classification Report of Test Set:")
table_data = [
    ["Sensitivity", "Specificity", "PPV", "NPV", "F1 Score"],
]

for i in range(len(TPR)):
    table_data.append([round(TPR[i], 2), round(TNR[i], 2), round(PPV[i], 2), round(NPV[i], 2), round(F1[i], 2)])

print(tabulate(table_data, headers="firstrow", tablefmt="pretty",showindex=True))